# 🧠  Document Q&A – Multimodal Document Pipeline (Streamlit UI)

This script powers an interactive Streamlit app that answers technical questions about Seclock’s door hardware documents using **multimodal AI**—combining vector search, LLM reasoning, and image validation via Snowflake Cortex.

## 🔧 High-Level Features

- 📥 **Document Ingestion & Processing**
  - Splits PDFs into per-page images and text using `parse_document()`.
  - Stores images in a Snowflake stage and text/metadata in structured tables.
  - Generates image embeddings using `embed_image_1024`.
---
- 🔍 **Semantic Search & Retrieval**
  - Converts user questions into temporary query images to leverage image embeddings.
  - Embeds the image and performs vector search via Cortex Search Service.
  - Retrieves top matching enriched chunks (text + metadata + image reference).
---
- 🧠 **LLM-Based Text Answering**
  - Feeds retrieved context into Claude (`claude-3-7-sonnet`) to generate:
    - Direct answers
    - Confidence scores
    - Justifications
    - Markdown-linked citations (`[Document - page X](presigned_url)`)
---
- 🖼️ **Image-Based Reasoning**
  - Identifies cited pages from the text answer and filters matching images.
  - Submits images asynchronously to Cortex for answer validation.
  - Uses image metadata and visual context to extract direct answers or critiques.
---
- 🧪 **Answer Synthesis**
  - Combines text and image-based answers into a final, human-readable response.
  - Rewrites for clarity, accuracy, and directness.
  - Prioritizes newer or more reliable sources and flags conflicting evidence.
  - Provides full citation trail with links to specific document pages.
---
- 🔗 **Embedding & Retrieval Enrichment**
  - Merges document metadata, summaries, page-level chunks, and vectors into a retrieval-optimized format.
  - Allows precise filtering and future expansion (e.g., by product line or brand).
---
- 💬 **Streamlit Chat UI**
  - Provides an interactive chatbot interface with step-by-step progress feedback.
  - Includes expandable debug sections for:
    - Source documents used
    - Text answer reasoning
    - Image-based critiques
    - LLM prompt previews


In [ ]:
# Import python packages
import os
import sys
import json
import shutil
import datetime
import re
import time
import hashlib
from difflib import SequenceMatcher
import tempfile
from textwrap import dedent
import streamlit as st
from PIL import Image, ImageDraw, ImageFont
from concurrent.futures import ThreadPoolExecutor, as_completed
from contextlib import contextmanager
from dataclasses import dataclass
from typing import List
from typing import Tuple
import snowflake.snowpark.session as session
import pdfplumber
import PyPDF2
import streamlit as st
from snowflake.snowpark.context import get_active_session
from snowflake.core import Root
from snowflake.cortex import complete, CompleteOptions
sp_session = get_active_session()

## 📄 PDF Preprocessing Pipeline for Document Analysis

Preprocesses PDFs stored in a Snowflake stage, preparing them for downstream AI document analysis. It lists all PDF files in a specified input stage, downloads each file temporarily, and performs two key operations: 

1. Splitting the PDF into individual pages and uploading each as a separate PDF file
2. Converting each page into a high-resolution image, optionally scaled to a maximum dimension, and uploading the images back to a specified output stage. 

In [ ]:
def print_info(msg: str) -> None:
    """Print info message"""
    print(f"INFO: {msg}", file=sys.stderr)


def print_error(msg: str) -> None:
    """Print error message"""
    print(f"ERROR: {msg}", file=sys.stderr)
    if hasattr(st, "error"):
        st.error(msg)


def print_warning(msg: str) -> None:
    """Print warning message"""
    print(f"WARNING: {msg}", file=sys.stderr)


@dataclass
class Config:
    input_stage: str = "@CORTEX_SEARCH_TUTORIAL_DB.PUBLIC.DOC_REPO/"
    output_stage: str = (
        "@CORTEX_SEARCH_TUTORIAL_DB.PUBLIC.DOC_REPO/PARSED/"  # Base output stage without subdirectories
    )
    input_path: str = "pre_processed"
    output_pdf_path: str = "paged_pdf"
    output_image_path: str = "paged_image"
    allowed_extensions: List[str] = None
    max_dimension: int = 1500  # Maximum dimension in pixels before scaling
    dpi: int = 300  # Default DPI for image conversion

    def __post_init__(self):
        if self.allowed_extensions is None:
            self.allowed_extensions = [".pdf"]


class PDFProcessingError(Exception):
    """Base exception for PDF processing errors"""


class FileDownloadError(PDFProcessingError):
    """Raised when file download fails"""


class PDFConversionError(PDFProcessingError):
    """Raised when PDF conversion fails"""


@contextmanager
def managed_temp_file(suffix: str = None) -> str:
    """Context manager for temporary file handling"""
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=suffix)
    try:
        yield temp_file.name
    finally:
        # Don't delete the file immediately, let the caller handle cleanup
        pass


def cleanup_temp_file(file_path: str) -> None:
    """Clean up a temporary file"""
    try:
        if os.path.exists(file_path):
            os.unlink(file_path)
    except OSError as e:
        print_warning(f"Failed to delete temporary file {file_path}: {e}")


def list_pdf_files(session: session.Session, config: Config) -> List[dict]:
    """List all PDF files in the source stage"""
    try:
        # Use LIST command instead of DIRECTORY function
        query = f"""
        LIST {config.input_stage}
        """

        file_list = session.sql(query).collect()

        # Filter for PDF files
        pdf_files = []
        for file_info in file_list:
            full_path = file_info["name"]
            # Extract just the filename from the full path
            file_name = os.path.basename(full_path)

            if any(
                file_name.lower().endswith(ext) for ext in config.allowed_extensions
            ):
                pdf_files.append(
                    {
                        "RELATIVE_PATH": file_name,  # Use just the filename
                        "FULL_STAGE_PATH": full_path,  # Use full path for download
                        "SIZE": file_info["size"] if "size" in file_info else 0,
                    }
                )

        print_info(f"Found {len(pdf_files)} PDF files in the stage")
        return pdf_files
    except Exception as e:
        print_error(f"Failed to list files: {e}")
        raise


def download_file_from_stage(
    session: session.Session, file_path: str, config: Config
) -> str:
    """Download a file from stage using session.file.get"""
    # Create a temporary directory
    temp_dir = tempfile.mkdtemp()
    try:
        # Ensure there are no double slashes in the path
        stage_path = f"{config.input_stage.rstrip('/')}/{file_path.lstrip('/')}"

        # Get the file from stage
        get_result = session.file.get(stage_path, temp_dir)
        if not get_result or get_result[0].status != "DOWNLOADED":
            raise FileDownloadError(f"Failed to download file: {file_path}")

        # Construct the local path where the file was downloaded
        local_path = os.path.join(temp_dir, os.path.basename(file_path))
        if not os.path.exists(local_path):
            raise FileDownloadError(f"Downloaded file not found at: {local_path}")

        return local_path
    except Exception as e:
        print_error(f"Error downloading {file_path}: {e}")
        # Clean up the temporary directory
        try:
            import shutil

            shutil.rmtree(temp_dir)
        except Exception as cleanup_error:
            print_warning(f"Failed to clean up temporary directory: {cleanup_error}")
        raise FileDownloadError(f"Failed to download file: {e}")


def upload_file_to_stage(
    session: session.Session, file_path: str, output_path: str, config: Config
) -> str:
    """Upload file to the output stage"""
    try:
        # Get the directory and filename from the output path
        output_dir = os.path.dirname(output_path)
        base_name = os.path.basename(output_path)

        # Create the full stage path with subdirectory
        stage_path = f"{config.output_stage.rstrip('/')}/{output_dir.lstrip('/')}"

        # Read the content of the original file
        with open(file_path, "rb") as f:
            file_content = f.read()

        # Create a new file with the correct name
        temp_dir = tempfile.gettempdir()
        temp_file_path = os.path.join(temp_dir, base_name)

        # Write the content to the new file
        with open(temp_file_path, "wb") as f:
            f.write(file_content)

        # Upload the file using session.file.put with compression disabled
        put_result = session.file.put(
            temp_file_path, stage_path, auto_compress=False, overwrite=True
        )

        # Check upload status
        if not put_result or len(put_result) == 0:
            raise Exception(f"Failed to upload file: {base_name}")

        if put_result[0].status not in ["UPLOADED", "SKIPPED"]:
            raise Exception(f"Upload failed with status: {put_result[0].status}")

        # Clean up the temporary file
        if os.path.exists(temp_file_path):
            os.remove(temp_file_path)

        return f"Successfully uploaded {base_name} to {stage_path}"
    except Exception as e:
        print_error(f"Error uploading file: {e}")
        raise


def process_pdf_files(config: Config) -> None:
    """Main process to orchestrate the PDF splitting"""
    try:
        session = get_active_session()
        pdf_files = list_pdf_files(session, config)

        for file_info in pdf_files:
            file_path = file_info["RELATIVE_PATH"]
            print_info(f"Processing: {file_path}")

            try:
                # Download the PDF file
                local_pdf_path = download_file_from_stage(session, file_path, config)

                # Get base filename without extension
                base_name = os.path.splitext(os.path.basename(file_path))[0]

                # Extract individual PDF pages
                with open(local_pdf_path, "rb") as file:
                    pdf_reader = PyPDF2.PdfReader(file)
                    num_pages = len(pdf_reader.pages)
                    print_info(f"Converting PDF to {num_pages} pages of PDFs")

                    for i in range(num_pages):
                        page_num = i + 1
                        s3_pdf_output_path = (
                            f"{config.output_pdf_path}/{base_name}_page_{page_num}.pdf"
                        )
                        pdf_writer = PyPDF2.PdfWriter()
                        pdf_writer.add_page(pdf_reader.pages[i])
                        temp_file = tempfile.NamedTemporaryFile(
                            delete=False, suffix=".pdf"
                        )
                        local_pdf_tmp_file_name = temp_file.name
                        with open(local_pdf_tmp_file_name, "wb") as output_file:
                            pdf_writer.write(output_file)
                        
                        upload_file_to_stage(
                            session, local_pdf_tmp_file_name, s3_pdf_output_path, config
                        )
                        cleanup_temp_file(local_pdf_tmp_file_name)
                            
                # Convert PDF to images                
                with pdfplumber.open(local_pdf_path) as pdf:
                    print_info(f"Converting PDF to {len(pdf.pages)} images")
                    for i, page in enumerate(pdf.pages):
                        page_num = i + 1
                        # Get page dimensions
                        width = page.width
                        height = page.height

                        # Determine if scaling is needed
                        max_dim = max(width, height)
                        if max_dim > config.max_dimension:
                            # Calculate scale factor to fit within max_dimension
                            scale_factor = config.max_dimension / max_dim
                            width = int(width * scale_factor)
                            height = int(height * scale_factor)

                        img = page.to_image(resolution=config.dpi)
                        temp_file = tempfile.NamedTemporaryFile(
                            delete=False, suffix=".png"
                        )
                        local_image_tmp_file_name = temp_file.name
                        img.save(local_image_tmp_file_name)

                        s3_image_output_path = (
                            f"{config.output_image_path}/{base_name}_page_{page_num}.png"
                        )
                        
                        upload_file_to_stage(
                            session, local_image_tmp_file_name, s3_image_output_path, config
                        )
                        cleanup_temp_file(local_image_tmp_file_name)
                        
                # Clean up the original downloaded file
                cleanup_temp_file(local_pdf_path)

            except Exception as e:
                print_error(f"Error processing {file_path}: {e}")
                continue

    except Exception as e:
        print_error(f"Fatal error in process_pdf_files: {e}")
        raise

In [ ]:
config = Config(dpi=300)
process_pdf_files(config)

## 🔍 Document Image Preview

To check everything has been processed as planned, we can look at an image representing a page from the PDFs.

In [ ]:
image_path = f"@CORTEX_SEARCH_TUTORIAL_DB.PUBLIC.DOC_REPO/PARSED/paged_image/2023-factbook_page_27.png"
try:
    image_st = sp_session.file.get_stream( image_path,decompress=False)
except Exception as e:
    print("failed to initialize file stream:", e)
    
image = image_st.read()
st.image(image)


In [ ]:
use role accountadmin

## 🧠 Batch Image Embedding with Cortex and Snowpark

This workflow performs batch image embedding using a Python stored procedure

1. **Identify Unprocessed Images**
   A temporary table (`limit_directory_table`) is created by listing all image files in the stage (`@utils.ai.stock_ikb_documents/paged_image/`) and filtering out those already embedded in the `output_vector_table`.

2. **Assign Row Numbers for Batching**
   Each unprocessed image file is assigned a `row_number()` so batches can be defined by row ranges (`start_rn` to `end_rn`).

3. **Define Embedding Procedure**
   A Python stored procedure `run_image_embedding_batch(start_rn, end_rn)` is created. It:

   * Reads a batch of image files from the temporary table.
   * Extracts file and metadata (e.g. file name, page number).
   * Computes an image embedding using `snowflake.cortex.embed_image_1024` with the `voyage-multimodal-3` model.
   * Saves the embeddings to `output_vector_table`.

4. **Queue Up Batch Jobs**
   The total number of batches is calculated, and a list of SQL `CALL` statements is built, one per batch.

5. **Run Jobs Concurrently**
   A loop manages job execution with up to 5 concurrent asynchronous jobs at a time. Each job is submitted using `.collect_nowait()` and polled until it completes.

6. **Monitor and Retry**
   Each batch is logged upon completion or failure, and the loop continues until all batches are processed.

This setup allows high-throughput embedding of images inside Snowflake, using Cortex's multimodal capabilities with minimal manual orchestration.


In [ ]:
create or replace procedure run_image_embedding_batch(start_rn int, end_rn int)
returns string
language python
runtime_version = 3.9
packages = ('snowflake-snowpark-python')
handler = 'embed_handler'
AS
$$
def embed_handler(session, start_rn, end_rn):
    df = session.sql(f'''
        select
            concat('paged_image/', split_part(relative_path, '/', -1)) as file_name,
            regexp_substr(file_name, 'paged_image/(.*)\\.png$', 1, 1, 'e', 1) as paged_file_name,
            split_part(paged_file_name, '_page_', 0) as original_file_name,
            split_part(paged_file_name, '_page_', 2)::int as page_number,
            '@CORTEX_SEARCH_TUTORIAL_DB.PUBLIC.DOC_REPO/' as stage_prefix,
            to_file(file_url)  as image_file,
            AI_EMBED(
                'voyage-multimodal-3', 
                '@CORTEX_SEARCH_TUTORIAL_DB.PUBLIC.DOC_REPO/'||concat('PARSED/paged_image/', split_part(relative_path, '/', -1))
            ) as image_vector
        from limit_directory_table
        where rn between {start_rn} and {end_rn}
    ''')
    ## print(df.columns)
    df.write.save_as_table("OUTPUT_VECTOR_TABLE", mode="append") ##change this to append
    return f"Embedded and saved RN {start_rn} to {end_rn}"
$$;

In [ ]:
--call run_image_embedding_batch(start_rn int, end_rn int)

In [ ]:
select get_ddl('table','output_vector_table');
create or replace TABLE OUTPUT_VECTOR_TABLE (
	FILE_NAME VARCHAR(16777216),
	PAGED_FILE_NAME VARCHAR(16777216),
	ORIGINAL_FILE_NAME VARCHAR(16777216),
	PAGE_NUMBER NUMBER(38,0),
	STAGE_PREFIX VARCHAR(43) NOT NULL,
	IMAGE_FILE FILE,
	IMAGE_VECTOR VECTOR(FLOAT, 1024)
);


In [ ]:
select * from output_vector_table

In [ ]:
BATCH_SIZE = 10
MAX_CONCURRENT = 5

# 1. Create LIMIT_DIRECTORY_TABLE if not exists
sp_session.sql("""
    create or replace temporary table limit_directory_table as
    select
        *,
        row_number() over (order by relative_path) as rn
    from
        directory(@CORTEX_SEARCH_TUTORIAL_DB.PUBLIC.DOC_REPO)
    where
        relative_path like '%paged_image/%'
        
""").collect()

    # create or replace temporary table limit_directory_table as
    # select
    #     *,
    #     row_number() over (order by relative_path) as rn
    # from
    #     directory(@CORTEX_SEARCH_TUTORIAL_DB.PUBLIC.DOC_REPO)
    # where
    #     relative_path like '%paged_image/%'
    #     and
    #     relative_path not in (
    #         select file_name from output_vector_table

### The above sql command should be used if we want to only load deltas
    #     

# 2. Get total batches
max_rn = sp_session.sql("select max(rn) AS max_rn from limit_directory_table").collect()[0]["MAX_RN"]
total_batches = (max_rn + BATCH_SIZE - 1) // BATCH_SIZE

# 3. Prepare all batch configs
batch_queue = []
for i in range(total_batches):
    start_rn = i * BATCH_SIZE + 1
    end_rn = min((i + 1) * BATCH_SIZE, max_rn)
    label = f"Batch {i+1}: RN {start_rn}-{end_rn}"
    sql = f"call run_image_embedding_batch({start_rn}, {end_rn})"
    batch_queue.append((sql, label))

# 4. Loop with max 5 concurrent jobs
active_jobs = []

while batch_queue or active_jobs:
    # Launch jobs if we have capacity
    while batch_queue and len(active_jobs) < MAX_CONCURRENT:
        sql, label = batch_queue.pop(0)
        print(f"🚀 Submitting async job for {label}")
        try:
            job = sp_session.sql(sql).collect_nowait()
            active_jobs.append((job, label))
        except Exception as e:
            print(f"❌ Failed to submit {label}: {e}")

    # Poll active jobs
    for job, label in active_jobs.copy():
        if job.is_done():
            try:
                result = job.result()
                print(f"✅ {label} completed: {result}")
            except Exception as e:
                print(f"❌ {label} failed: {e}")
            active_jobs.remove((job, label))

    if active_jobs:
        time.sleep(15)

print("🎉 All batches processed.")

In [ ]:
select count(*) from output_vector_table

## 🔖 Extract Text from PDF Pages

This SQL script creates a table (`pdf_pages`) that extracts and stores parsed text content from individual PDF pages:

1. **Filter Input Files**
   It queries the stage `@utils.ai.stock_ikb_documents` and filters files whose path matches the pattern `%paged_pdf/%`, meaning individual page PDFs from previously split documents.

2. **Extract File Metadata**
   For each PDF file:

   * `file_name` is constructed by prefixing the relative path with `paged_pdf/`.
   * `paged_file_name` extracts just the PDF filename using regex.
   * `original_file_name` removes the `_page_X` suffix to get the base document name.
   * `page_number` is parsed from the filename to track the page.

3. **Generate File References**
   The `to_file(file_url)` function creates a file object for use in Cortex functions.

4. **Parse PDF Content with Cortex**
   The `snowflake.cortex.parse_document` function is called on each page to extract its text layout. The result is cast to a string, then parsed as JSON and stored in the `pdf_text` column.

5. **Output the Resulting Table**
   The final table `pdf_pages` includes:

   * File path and name metadata
   * Page number
   * File reference
   * Structured PDF content parsed by Cortex

This process enables structured, searchable access to individual page-level text from large documents using Cortex's layout-aware parsing engine.


In [ ]:
create or replace table pdf_pages as
select
    concat('PARSED/paged_pdf/', split_part(relative_path, '/', -1)) as file_name,
    regexp_substr(file_name, 'PARSED/paged_pdf/(.*)\\.pdf$', 1, 1, 'e', 1) as paged_file_name,
    split_part(paged_file_name, '_page_', 0) as original_file_name,
    split_part(paged_file_name, '_page_', 2)::int as page_number,
    '@CORTEX_SEARCH_TUTORIAL_DB.PUBLIC.DOC_REPO' as stage_prefix,
    to_file(file_url) as pdf_file,
    parse_json(
        to_varchar(
            snowflake.cortex.parse_document(
                '@CORTEX_SEARCH_TUTORIAL_DB.PUBLIC.DOC_REPO',
                file_name,
                {'mode': 'LAYOUT'}
            )
        )
    ):content as pdf_text
from
    directory(@CORTEX_SEARCH_TUTORIAL_DB.PUBLIC.DOC_REPO)
where
    relative_path like '%paged_pdf/%'
;

In [ ]:
select * from pdf_pages;

In [ ]:
select * from output_vector_table;

## 📖 Enriching PDF Pages with Metadata and Text Chunks for Semantic Search

This SQL pipeline creates a comprehensive table (`utils.ai.pdf_images_joined`) that combines page-level text, image embeddings, structured metadata, and semantically formatted chunks optimized for multimodal document retrieval using Snowflake Cortex.

### ✅ Steps:

1. **🖇️ Join PDF Pages with Image Embeddings**

   * Merges parsed PDF page data from `pdf_pages` with vector embeddings from `output_vector_table` via `paged_file_name`.
---
2. **📄 Select Representative Pages for Metadata**

   * Uses `row_number()` to select:

     * The **first 10 pages** (for coverage of typical document headers).
     * The **last 2 pages** (often contain part indexes or summaries).
---
3. **🧠 Generate Document-Level Metadata**

   * Concatenates the selected pages’ text and feeds it into `ai_complete()` (with `llama4-scout`) to extract:

     * `manufacturer`
     * `product_line`
     * `document_type`
     * `effective_date`, `copyright`
     * `category`
     * `concise_document_summary`
---
4. **📝 Generate Page-Level Metadata**

   * Runs `ai_complete()` (with `llama4-scout`) on each page’s text to extract:

     * `page_title`
     * `concise_page_summary`
---
5. **🔗 Join Metadata with Full Page Content**

   * Combines document-level and page-level metadata with:

     * Raw page text
     * Vector embeddings
     * File references
---
6. **✂️ Split Pages into Chunks**

   * Uses `cortex.split_text_recursive_character()` to break page text into \~1800-character, markdown-safe blocks, ensuring semantic cohesion for chunk-level retrieval.
---
7. **🔍 Enrich Chunks with Visual Context**

   * For each chunk:

     * Runs `ai_complete()` (with `llama4-maverick`) using the **full page image** and **chunk text**.
     * Extracts structured visual context such as:

       * **Page region**
       * **Table sections or headers**
       * **Related elements not captured in the chunk**
     * Encourages bullet-point or key-value output grounded in visual layout.
---
8. **🧱 Build Final Enriched Chunks**

   * Combines:

     * Source file
     * Document and page metadata
     * Chunk visual context
     * Raw chunk text
   * Stores final result in an `enriched_chunk` field optimized for LLM prompts and semantic indexing.

In [ ]:
 select
        pdfs.file_name as pdf_file_name,
        images.file_name as image_file_name,
        pdfs.original_file_name,
        pdfs.page_number,
        pdfs.pdf_file,
        images.image_file,
        images.image_vector,
        pdfs.pdf_text,
    from
        pdf_pages as pdfs
    join
        output_vector_table as images
        on
        pdfs.paged_file_name = images.paged_file_name

In [ ]:

create or replace table pdf_images_joined as
with pdf_images_joined as (
    select
        pdfs.file_name as pdf_file_name,
        images.file_name as image_file_name,
        pdfs.original_file_name,
        pdfs.page_number,
        pdfs.pdf_file,
        images.image_file,
        images.image_vector,
        pdfs.pdf_text,
    from
        pdf_pages as pdfs
    join
        output_vector_table as images
        on
        pdfs.paged_file_name = images.paged_file_name
),
first_10_pages as (
  select
    original_file_name,
    page_number,
    pdf_text,
    row_number() over (partition by original_file_name order by page_number) as row_num_start,
    row_number() over (partition by original_file_name order by page_number desc) as row_num_end
  from
    pdf_images_joined
),
limited_pages as (
  select
    original_file_name,
    page_number,
    pdf_text
  from 
    first_10_pages
  where 
    row_num_start <= 10
    or
    row_num_end <= 2
),
document_text as (
  select
    original_file_name,
    listagg(pdf_text, '\n\n') within group (order by page_number) as full_text
  from 
    limited_pages
  group by 
    original_file_name
),
get_document_summary as (
    select
      original_file_name,
      full_text,
      ai_complete(
        model => 'claude-4-sonnet',
        prompt => concat(
          'You are analyzing the 2023 Investment Company Institute (ICI) Fact Book, the definitive statistical compendium of the US investment company industry. ',
          'This document contains authoritative data on mutual funds, ETFs, closed-end funds, and other registered investment companies.\n\n',
          
          'CRITICAL ICI FACT BOOK CONTEXT:\n',
          '- Total industry assets: ~$27+ trillion across all investment company types\n',
          '- Time coverage: Multi-year trend data typically spanning 10+ years ending 2023\n',
          '- Geographic scope: Primarily US registered investment companies with some global context\n',
          '- Data authority: Official industry statistics used by regulators, researchers, and investment professionals\n\n',
          
          'EXTRACTION FOCUS - Identify these key document characteristics:\n',
          '1. ASSET UNIVERSE: What types of investment vehicles are covered (mutual funds, ETFs, closed-end, etc.)\n',
          '2. DATA SCOPE: Geographic coverage (US domestic, international, global)\n',
          '3. TIME RANGE: Years covered for trend analysis and current statistics\n',
          '4. STATISTICAL CATEGORIES: Asset classes, fund types, market segments analyzed\n',
          '5. REGULATORY CONTEXT: SEC regulations, industry standards, compliance frameworks\n',
          '6. MARKET ANALYSIS: Flow data, performance metrics, expense ratios, market concentration\n\n',
          
          'PRECISION REQUIREMENTS:\n',
          '- Use exact ICI terminology (e.g., "registered investment companies", "net assets", "total net flows")\n',
          '- Specify data years and time periods precisely\n',
          '- Distinguish between asset classes vs. fund types vs. investment objectives\n',
          '- Note geographic scope explicitly (US vs. worldwide data)\n',
          '- Identify statistical methodologies and data sources\n\n',
          
          'OUTPUT FORMAT - Extract only these fields if confidently identified:\n',
          array_to_string(
              array_construct(
                  'document_type',
                  'publication_year', 
                  'primary_data_years_covered',
                  'geographic_scope',
                  'investment_company_types_included',
                  'asset_classes_analyzed', 
                  'key_statistical_measures',
                  'regulatory_framework_context',
                  'industry_trend_timeframes',
                  'data_source_authority'
              ),
              '\n\t* '
          ),
          '\n\nRules:\n',
          '1. Use ICI-standard terminology and precise financial language\n',
          '2. Be specific about time periods (e.g., "2014-2023 trend analysis")\n',
          '3. Distinguish between different types of financial metrics\n',
          '4. Only include information with high confidence\n',
          '5. Return ONLY key:value pairs, no additional text\n\n',
          'Document content:\n', full_text, '\n\n'
        ),
        model_parameters => {
          'temperature': 0.1,
          'max_tokens': 1500
        }
      )::string as document_metadata
    from 
        document_text
),
describe_pages as (
    select
        pdf_file_name,
        image_file_name,
        original_file_name,
        page_number,
        pdf_file,
        image_file,
        image_vector,
        pdf_text,
        ai_complete(
          model => 'claude-4-sonnet',
          prompt => concat(
            'You are analyzing individual pages from the 2023 ICI Investment Company Fact Book. Each page contains specific financial data, statistics, charts, or analysis segments that serve distinct research and analytical purposes.\n\n',
            
            'ICI PAGE ANALYSIS EXPERTISE:\n',
            '- Pages typically focus on specific data themes: asset allocation, fund flows, expense analysis, performance metrics, market trends\n',
            '- Statistical tables show precise numerical data with time series\n',
            '- Charts visualize trends, comparisons, and distributions\n',
            '- Text sections provide context, methodology, and interpretation\n',
            '- Footnotes contain critical definitional and methodological information\n\n',
            
            'PAGE-LEVEL EXTRACTION FOCUS:\n',
            '1. PRIMARY DATA THEME: What specific aspect of investment company data is the main focus?\n',
            '2. FINANCIAL METRICS: Exact types of measurements (assets, flows, returns, ratios, percentages)\n',
            '3. TIME DIMENSION: Specific years, quarters, or time periods covered on this page\n',
            '4. MARKET SEGMENTATION: Fund types, asset classes, geographic regions, or investor categories\n',
            '5. VISUAL ELEMENTS: Types of charts, tables, or data presentations\n',
            '6. QUANTITATIVE SCOPE: Scale of data (billions, trillions, percentages, basis points)\n\n',
            
            'ICI-SPECIFIC PATTERN RECOGNITION:\n',
            '- Asset allocation pages: Equity, fixed income, money market, hybrid breakdowns\n',
            '- Flow analysis pages: Net flows, inflows, outflows by fund type or time period\n',
            '- Performance pages: Returns, volatility, benchmarking data\n',
            '- Market structure pages: Concentration, market share, competitive dynamics\n',
            '- Expense analysis pages: Fee structures, expense ratios, cost trends\n',
            '- Demographic pages: Investor characteristics, distribution channels\n\n',
            
            'PRECISION REQUIREMENTS:\n',
            '- Identify specific ICI data categories and subcategories\n',
            '- Note exact time periods referenced (year-end vs. quarterly vs. cumulative)\n',
            '- Distinguish between gross and net measures, flows vs. assets vs. returns\n',
            '- Specify if data is US-only, international, or global\n',
            '- Identify footnotes or methodological qualifiers\n\n',
            
            'OUTPUT FIELDS - Extract only if clearly present:\n',
            array_to_string(
              array_construct(
                'primary_data_focus',
                'specific_financial_metrics',
                'time_periods_covered', 
                'fund_types_or_asset_classes',
                'geographic_scope_if_specified',
                'visual_presentation_type',
                'key_quantitative_highlights',
                'methodological_notes_if_present'
              ),
              '\n\t* '
            ), '\n\n',
            
            'Rules:\n',
            '1. Use precise ICI terminology and financial language\n',
            '2. Be specific about data categories and time periods\n',
            '3. Focus on what makes this page unique within the larger document\n',
            '4. Note visual elements that would aid in retrieval\n',
            '5. Return ONLY key:value pairs\n\n',
            'Page content:\n', pdf_text, '\n\n'
          ),
          model_parameters => {
            'temperature': 0.05,
            'max_tokens': 1200
          }
        )::string as page_metadata
    from
        pdf_images_joined
),
pages_with_metadata as (
  select
    page.pdf_file_name,
    page.image_file_name,
    page.original_file_name,
    page.page_number,
    page.pdf_file,
    page.image_file,
    page.image_vector,
    page.pdf_text,
    page.page_metadata,
    doc.document_metadata
  from
    describe_pages page
  join
    get_document_summary doc
    on 
        page.original_file_name = doc.original_file_name
),
split_pages_into_chunks as (
    select
        pdf_file_name,
        image_file_name,
        original_file_name,
        page_number,
        image_vector,
        pdf_text,
        document_metadata,
        page_metadata,
        ai_complete(
            model => 'claude-4-sonnet',
            predicate => concat(
                'You are performing multimodal analysis of ICI Investment Company Fact Book content, analyzing both visual page images and extracted text chunks to create enriched context for financial data retrieval.\n\n',
                
                'MULTIMODAL ANALYSIS OBJECTIVE:\n',
                'Describe how this specific text chunk relates to the visual elements (tables, charts, graphs) and overall financial data presentation in the page image, creating searchable context for investment industry professionals.\n\n',
                
                'ICI FACT BOOK VISUAL-TEXT INTEGRATION PATTERNS:\n\n',
                
                'FOR STATISTICAL TABLES:\n',
                '- Text chunk position: header row, data row, footnote, or summary section\n',
                '- Data hierarchy: main category, subcategory, or detailed breakdown\n',
                '- Temporal context: specific year, time series position, or trend indicator\n',
                '- Cross-references: table numbers, figure citations, or related data points\n',
                '- Quantitative context: units (billions, percentages), scale factors, precision levels\n\n',
                
                'FOR CHARTS AND GRAPHS:\n',
                '- Visual relationship: axis label, data series, legend item, or chart title\n',
                '- Data representation: trend line point, category segment, or comparative element\n',
                '- Time series position: starting point, endpoint, peak/trough, or inflection point\n',
                '- Category classification: fund type, asset class, geographic region, or market segment\n',
                '- Performance indicators: growth rates, market share changes, or volatility measures\n\n',
                
                'FOR NARRATIVE AND ANALYSIS:\n',
                '- Data interpretation: statistical finding explanation, trend analysis, or market insight\n',
                '- Methodology context: calculation method, data source, or measurement standard\n',
                '- Industry context: regulatory impact, market dynamic, or competitive factor\n',
                '- Forward-looking elements: projections, implications, or industry outlook\n',
                '- Comparative analysis: benchmarking, historical context, or peer comparisons\n\n',
                
                'ENHANCED CONTEXT EXTRACTION:\n',
                '1. QUANTITATIVE PRECISION: Extract exact figures, percentages, time periods with proper units and context\n',
                '2. VISUAL POSITIONING: Describe where this chunk appears in charts, tables, or visual hierarchies\n',
                '3. ICI TERMINOLOGY: Use standard investment industry language and ICI-specific categorizations\n',
                '4. SEARCHABILITY: Include keywords that financial professionals would use to find this data\n',
                '5. CROSS-REFERENCES: Note connections to other data points, charts, or analytical sections\n\n',
                
                'FINANCIAL DATA CONTEXTUALIZATION:\n',
                '- Asset allocation context: Which asset classes, geographic regions, or fund types\n',
                '- Flow analysis context: Inflows vs. outflows, net flows, seasonal patterns\n',
                '- Performance context: Returns, volatility, risk-adjusted measures, benchmarking\n',
                '- Market structure context: Concentration, market share, competitive dynamics\n',
                '- Cost analysis context: Expense ratios, fee structures, cost trends over time\n',
                '- Regulatory context: Compliance requirements, reporting standards, rule impacts\n\n',
                
                'OUTPUT FORMAT:\n',
                '**Visual Context**: [How this text relates to charts, tables, or visual elements]\n',
                '**Data Classification**: [ICI category, fund type, asset class, or market segment]\n',
                '**Quantitative Details**: [Specific figures, time periods, units, and scale]\n',
                '**Search Keywords**: [Terms financial professionals would use to find this data]\n',
                '**Cross-References**: [Related data points, charts, or sections]\n',
                '**Industry Relevance**: [Why this data matters for investment analysis or research]\n\n',
                
                'PRECISION REQUIREMENTS:\n',
                '- Use exact ICI terminology and standard industry language\n',
                '- Maintain quantitative precision with proper units and time qualifiers\n',
                '- Create multiple pathways for data discovery (fund type, asset class, time period, etc.)\n',
                '- Link visual elements to searchable concepts\n',
                '- Focus on actionable insights for financial analysis\n\n',
                
                'Text chunk to analyze:\n', value::string, '\n\n'
            ),
            file => image_file,
            model_parameters => {
              'temperature': 0.1,
              'max_tokens': 1500
            }
        )::string as chunk_context,
        concat(
            '**Source File:** ', original_file_name, '\n',
            '**Document Metadata:**\n', coalesce(document_metadata, 'N/A'), '\n\n',
            '===========================================\n\n',
            '**Page Metadata:**\n', coalesce(page_metadata, 'N/A'), '\n',
            '**Page Number:** ', page_number::string, '\n\n',
            '===========================================\n\n',
            '**Multimodal Chunk Analysis:**\n', chunk_context,
             '\n\n===========================================\n\n',
            '**Original Text Chunk:**\n\n', value::string
        ) as enriched_chunk,
        -- NEW: Store raw chunk text for hybrid search capability
        value::string as raw_chunk_text
    from
        pages_with_metadata,
    lateral flatten(
        input=>snowflake.cortex.split_text_recursive_character(
            pdf_text,
            'markdown',
            1800,
            200
        )
    )
)
select
    pdf_file_name,
    image_file_name,
    original_file_name,
    page_number,
    image_vector,
    pdf_text,
    enriched_chunk,
    raw_chunk_text
from
    split_pages_into_chunks



In [ ]:
select * from pdf_images_joined;

update  pdf_images_joined
 set image_file_name='PARSED/'||IMAGE_FILE_NAME;

## 🕵️‍♀️ Build Cortex Search Service

We're using the [User-Provided Vector Embeddings in Cortex Search](https://docs.snowflake.com/LIMITEDACCESS/cortex-search/user-provided-vectors) private preview. This allows us to provide precomputed vector embeddings to index and query with Cortex Search, which allows us to use our image embeddings as part of the search service.

In [ ]:


-- ENHANCED HYBRID SEARCH SERVICE - Indexes both raw and enriched text
create or replace cortex search service docs_search_service
    text indexes (pdf_text,enriched_chunk,raw_chunk_text)
    vector indexes (image_vector)
    warehouse = CORTEX_SEARCH_TUTORIAL_WH
    target_lag = '1 day'
    as 
    select 
        pdf_file_name,
        image_file_name,
        original_file_name,
        page_number,
        image_vector,
        pdf_text::varchar as pdf_text,
        enriched_chunk,
        raw_chunk_text
    from 
       pdf_images_joined
;



## 🤖 Multimodal Document Question Answering with Image-Aided Semantic Search

This system answers precise technical questions about door hardware products using both **text** and **image-based** reasoning across PDFs. It completes a three-step multimodal process:

1. **Text-Based Answering** using enriched document chunks.
2. **Image-Based Answer Validation** via OCR page images.
3. **Answer Synthesis** to revise and finalize the response.

This is done using the following steps:

### 1. **Convert User Question into Image Embedding**

* `get_text_embedding_via_image()`:

  * Renders the user's question as a temporary PNG image.
  * Uploads it to a Snowflake stage (`@utils.ai.stock_ikb_documents/queries/`).
  * Uses `snowflake.cortex.embed_image_1024()` to generate a **multimodal embedding** via `voyage-multimodal-3`.

---

### 2. **Perform Semantic Search with Text + Embedding**

* `query_search_service()`:

  * Submits both the raw question and the image embedding to a **Cortex search service**.
  * Returns the top 50 `ENRICHED_CHUNK`s (text + metadata + image references) relevant to the question.

---

### 3. **Rephrase Question for Search Compatibility**

* `rephrase_for_search()`:

  * Normalizes the user query by trimming and lowercasing it.
  * Helps align question formatting with indexed content.

---

### 4. **Build Context and Generate Text-Based Answer**

* `ai_complete_on_text()`:

  * Constructs a markdown-rich prompt from the top retrieved chunks.
  * Includes clickable links to the original PDF pages via `presigned_url`.
  * Uses `claude-3-7-sonnet` to answer the question directly and include:

    * A clear **direct answer**,
    * A **confidence score** (0–1),
    * A short **justification**, and
    * Properly formatted **CITED SOURCES**:
      [`Document Name - page X`](presigned_url)

---

### 5. **Deduplicate and Filter Pages for Image Critique**

* Duplicates are removed based on `(PDF_FILE_NAME, IMAGE_FILE_NAME)` pairs.
* `extract_cited_docs_and_pages()` ensures only **relevant document pages** are processed.
* `extract_page_number()` helps match images to cited pages.

---

### 6. **Run Image-Based Validations (Async)**

* `ai_complete_on_image_async()`:

  * Submits each matched image to `claude-3-7-sonnet` via `ai_complete()` using:

    * Document metadata
    * Page number
    * The original text answer for critique
  * Prompt instructs the model to confirm or revise the answer, ensuring it's grounded in the visual page content.

* `resolve_async_job()`:

  * Polls the result and extracts fields like:

    * `RESULT`, `PAGE_NUMBER`, `IMAGE_FILE_NAME`, `PRESIGNED_URL`.

---

### 7. **Filter Image-Based Answers by Confidence**

* `filter_by_confidence()`:

  * Retains only image completions with `CONFIDENCE >= 0.5`.
  * Helps ensure only high-quality critiques contribute to the final answer.

---

### 8. **Synthesize the Final Answer for the User**

* `synthesise_all_answers()`:

  * Merges text and image critiques into a unified prompt.
  * Final LLM prompt includes:

    * Original text result
    * All image critiques (linked to page images)
  * The LLM is instructed to:

    * Revise or reaffirm the answer
    * Rephrase for user clarity (customer-facing tone)
    * Exclude technical fields like `CONFIDENCE` or `JUSTIFICATION`
    * Append a **"Cited Sources"** section with markdown hyperlinks

---

In [ ]:
# def query_search_service(session, my_service, query_text):
#     query_embedding = get_text_embedding_via_image(session, query_text)
#     resp = my_service.search(
#         query = query_text,
#         experimental = {
#             "QueryEmbedding": query_embedding
#         },
#         columns=[
#             "ENRICHED_CHUNK",
#             "PDF_FILE_NAME",
#             "IMAGE_FILE_NAME",
#             "ORIGINAL_FILE_NAME",
#             "PAGE_NUMBER"
#         ],
#         limit=5
#     )
#     return resp.to_json()
 

def query_multi_index_search_service(session, my_service, query_text):
    """ENHANCED HYBRID SEARCH: Image + Enriched Text + Raw Text"""
    query_embedding = get_text_embedding_via_image(session, query_text)
    
    resp = my_service.search(
        # Use ONLY multi_index_query, not both query and multi_index_query
        multi_index_query={
            "image_vector": [
                {"vector": query_embedding}],
             "enriched_chunk":[{"text":query_text}],
            "pdf_text":[{"text":query_text}],
            "raw_chunk_text":[{"text":query_text}]},
        
        columns=[
            "ENRICHED_CHUNK",
            "RAW_CHUNK_TEXT",
            "PDF_FILE_NAME",
            "IMAGE_FILE_NAME",
            "ORIGINAL_FILE_NAME",
            "PAGE_NUMBER"
        ],
        limit=1000
    )
    
    return resp.to_json() 

# def query_multi_index_search_service(session, my_service, query_text):
#     query_embedding = get_text_embedding_via_image(session, query_text)
    
#     resp = my_service.search(
#         # Use ONLY multi_index_query, not both query and multi_index_query
#         multi_index_query={
#             "image_vector": [
#                 {"vector": query_embedding}
#             ],
#         },
        
#         columns=[
#             "ENRICHED_CHUNK",
#             "PDF_FILE_NAME",
#             "IMAGE_FILE_NAME",
#             "ORIGINAL_FILE_NAME",
#             "PAGE_NUMBER"
#         ],
#         limit=20
#     )
    
#     return resp.to_json()

def create_temp_image_from_text(text: str) -> tuple[str, str]:
    query_hash = hashlib.md5(text.strip().lower().encode()).hexdigest()
    image_filename = f"{query_hash}.png"

    temp_file = tempfile.NamedTemporaryFile(suffix=".png", delete=False)
    file_path = temp_file.name
    temp_file.close()

    image = Image.new("RGB", (1000, 200), "white")
    draw = ImageDraw.Draw(image)
    font = ImageFont.load_default()
    draw.text((10, 10), text, fill="black", font=font)
    image.save(file_path)


    return file_path, image_filename


def extract_cited_docs_and_pages(text_answer_str):
    cited = {}
    
    # Find the CITED SOURCES section (more robust extraction)
    cited_section_match = re.search(r"CITED SOURCES:\s*(.+?)(?:\n\n|$)", text_answer_str, re.IGNORECASE | re.DOTALL)
    
    if cited_section_match:
        cited_section = cited_section_match.group(1)
        print(f"DEBUG: Found cited section: {cited_section[:200]}...")
        
        # Pattern 1: [document - page X](url) format
        doc_page_pairs = re.findall(r"\[([a-zA-Z0-9._ -]+?)\s*-\s*page\s*(\d+)\]", cited_section)
        for doc, page in doc_page_pairs:
            doc = doc.strip().lower()
            page = page.strip()
            cited.setdefault(doc, set()).add(page)
            print(f"DEBUG: Added citation: {doc} -> page {page}")
        
        # Pattern 2: [document](url) format - fallback for documents without explicit pages
        if not doc_page_pairs:
            doc_links = re.findall(r"\[([a-zA-Z0-9._ -]+?)\]\(", cited_section)
            for doc in doc_links:
                doc_clean = doc.strip().lower()
                cited.setdefault(doc_clean, set()).add("*")  # Wildcard for any page
                print(f"DEBUG: Added wildcard citation: {doc_clean} -> *")
    else:
        print("DEBUG: No CITED SOURCES section found in text")
    
    print(f"DEBUG: Final extracted citations: {cited}")
    return cited

def extract_page_number(image_file_name: str) -> str:
    match = re.search(r'_page_(\d+)\.png$', image_file_name)
    return match.group(1) if match else "N/A"


def file_exists_in_stage(session, stage_name: str, file_path: str) -> bool:
    result = session.sql(f"list @{stage_name}/{file_path}").collect()
    return bool(result)


def fuzzy_match(a, b, threshold=0.6):
    return SequenceMatcher(None, a.lower(), b.lower()).ratio() >= threshold


def upload_file_to_stage(session, local_path: str, stage_name: str, dest_file_name: str):
    temp_dir = tempfile.gettempdir()
    temp_named_path = os.path.join(temp_dir, dest_file_name)

    os.makedirs(os.path.dirname(temp_named_path), exist_ok=True)
    shutil.copyfile(local_path, temp_named_path)

    try:
        result = session.file.put(
            temp_named_path,
            f"@{stage_name}/queries",
            overwrite=True,
            auto_compress=False
        )
        
    finally:
        os.remove(temp_named_path)


def get_text_embedding_via_image(
    session, 
    text: str, 
    stage_name="@cortex_search_tutorial_db.public.doc_repo"
):
    temp_path, image_filename = create_temp_image_from_text(text)
    stage_subpath = f"queries/{image_filename}"

    try:
        if not file_exists_in_stage(session, stage_name.lstrip("@"), stage_subpath):
            upload_file_to_stage(session, temp_path, stage_name.lstrip("@"), stage_subpath)
            
        query = f"""
            select 
                AI_EMBED(
                    'voyage-multimodal-3', 
                    '{stage_name}+{stage_subpath.lstrip('/')}'
                )
        """
        embedding = session.sql(query).collect()[0][0]
    finally:
        os.remove(temp_path)

    return embedding


def resolve_async_job(job):
    try:
        row = job.result()[0].asDict()
        return {
            "RESULT": row["RESULT"],
            "ORIGINAL_FILE_NAME": row["ORIGINAL_FILE_NAME"],
            "IMAGE_FILE_NAME": row["IMAGE_FILE_NAME"],
            "PRESIGNED_URL": row.get("PRESIGNED_URL", "#")
        }
    except Exception as e:
        return {
            "RESULT": f"Error: {e}",
            "ORIGINAL_FILE_NAME": None,
            "IMAGE_FILE_NAME": None,
            "PRESIGNED_URL": "#"
        }


def rephrase_for_search(question):
    return question.strip().lower()


def filter_by_confidence(responses, threshold=0.5):
    filtered = []
    for item in responses:
        match = re.search(r"CONFIDENCE:\s*([0-9.]+)", item["RESULT"], re.IGNORECASE)
        score = float(match.group(1)) if match else 0.0
        if score >= threshold:
            filtered.append(item)
    return filtered


def sql_escape(value):
    return str(value).replace("'", "''") if value is not None else ""


def run_model(model_name, llm_prompt, session, temperature, max_tokens, top_p, guardrails, stream):
    return complete(
        model=model_name,
        prompt=[{"role": "user", "content": llm_prompt}],
        session=session,
        options=CompleteOptions(
            temperature=temperature,
            max_tokens=max_tokens,
            top_p=top_p,
            guardrails=guardrails
        ),
        stream=stream
    )


def ai_complete_on_text(session, question, retrieved_chunks):
    seen = set()
    enriched_context_blocks = []

    for chunk in retrieved_chunks:
        enriched_chunk = chunk["ENRICHED_CHUNK"]
        original_file = chunk.get("ORIGINAL_FILE_NAME")
        image_file = chunk.get("IMAGE_FILE_NAME")

        if not image_file or not original_file:
            continue

        key = (original_file, image_file, enriched_chunk)
        if key in seen:
            continue
        seen.add(key)

        # Generate presigned URL
        presigned_url = session.sql(
            f"SELECT GET_PRESIGNED_URL(@cortex_search_tutorial_db.public.doc_repo, '{sql_escape(image_file)}')"
        ).collect()[0][0]

        # Format for the model
        block = dedent(f"""
        ---
        📄 **Source**: [{original_file}]({presigned_url})
        📜 **Extracted Content**:
        {enriched_chunk}
        """).strip()

        enriched_context_blocks.append(block)

    if not enriched_context_blocks:
        return {"result": "No usable context.", "metadata": {}}

    full_context = "\n\n".join(enriched_context_blocks)

    prompt = dedent(f"""
    You are an expert analyst of the 2023 Investment Company Institute (ICI) Fact Book, a comprehensive 
    statistical compendium containing precise financial data about US and global investment companies.

    ## 2023 ICI FACT BOOK SPECIFIC INTELLIGENCE:

    ### Known Data Context:
    - Total registered investment company assets: ~$27+ trillion as of year-end 2023
    - Breakdown by: Mutual Funds (~$20+ trillion), ETFs (~$6+ trillion), Closed-End (~$200+ billion)
    - Major asset classes: Equity (domestic/international), Fixed Income, Money Market, Hybrid
    - Key trend timeframes: 2019-2023 (5-year), 2014-2023 (10-year)

    ### Critical Terminology Precision:
    - "Net assets" = Assets minus liabilities (standard ICI metric)
    - "Total net assets" = Sum across all fund types unless qualified
    - "Asset allocation" = Investment portfolio composition by asset class
    - "Fund assets" = Assets within specific fund type only
    - "Investment company assets" = All registered funds combined

    ## ICI FACT BOOK DATA ARCHITECTURE:
    
    ### Asset Classification Hierarchy:
    - **LEVEL 1 - Asset Classes**: Equity, Fixed Income, Money Market, Hybrid/Balanced
    - **LEVEL 2 - Geographic Scope**: Domestic, International, Global, Regional
    - **LEVEL 3 - Investment Vehicles**: Mutual Funds, ETFs, Closed-End Funds
    - **LEVEL 4 - Investment Objectives**: Growth, Value, Blend, Sector-Specific, Target-Date
    
    ### Data Presentation Standards:
    - **Net Assets**: Always in billions of dollars unless specified otherwise
    - **Percentages**: Typically represent share of total within category
    - **Time Periods**: Year-end data (December 31) unless noted as quarterly
    - **Geographic Coverage**: US data unless explicitly marked as "Worldwide"
    - **Fund Universe**: All registered investment companies unless subset specified
    
    ### Visual Data Types in Context:
    - **Figure Tables**: Numerical data in structured rows/columns with precise values
    - **Bar Charts**: Year-over-year comparisons, often 5-10 year timeframes  
    - **Pie Charts**: Percentage breakdowns that sum to 100%
    - **Line Graphs**: Trend analysis over multiple years
    - **Flow Charts**: Net flows (inflows minus outflows) in billions

    ## PRECISION GUARDRAILS:

    ### Red Flag Validation Checks:
    1. **Scale Reasonableness**: US mutual fund assets should be $15-25 trillion range
    2. **Percentage Validation**: Asset allocation percentages must sum to ~100%
    3. **Temporal Consistency**: 2023 data should show logical progression from 2022
    4. **Geographic Logic**: US domestic equity typically 40-60% of total equity assets
    5. **Fund Type Ratios**: Mutual funds typically 3-4x larger than ETF assets

    ## CRITICAL INTERPRETATION RULES:
    
    ### Asset Allocation Questions:
    - "Net investments by asset class" = TOP-LEVEL asset allocation across equity/fixed income/money market/hybrid
    - "Total net assets" = Sum across ALL investment company types (mutual funds + ETFs + closed-end)
    - "Asset allocation" WITHOUT qualifiers = Comprehensive breakdown across all major categories
    - "Fund assets" WITH qualifiers = Specific to mentioned fund type only
    
    ### Temporal Context:
    - Always specify data year (2023, 2022, etc.)
    - Note if data is year-end vs. quarterly vs. cumulative
    - Multi-year questions require trend analysis across time periods
    
    ### Scale and Scope Precision:
    - Billions vs. trillions notation matters
    - US-only vs. global data distinction is critical
    - Registered vs. unregistered investment companies
    - Retail vs. institutional share classes

    ## INTELLIGENT QUESTION ROUTING:

    ### Question Pattern Analysis:
    - **Allocation Questions** ("by asset class", "breakdown", "distribution")
      → Expect percentage outputs from comprehensive data tables
    - **Trend Questions** ("growth", "change", "over time")  
      → Expect directional analysis from time series data
    - **Comparison Questions** ("vs", "compared to", "relative")
      → Expect relative metrics from comparative charts
    - **Scale Questions** ("total", "size", "how much")
      → Expect absolute values from aggregate statistics

    ## ENHANCED ACCURACY PROTOCOLS:
    
    ### Data Validation Checklist:
    1. **Scope Match**: Does data scope exactly match question parameters?
    2. **Time Alignment**: Is the time period precisely what was asked?
    3. **Scale Verification**: Are units (billions/percentages) correctly interpreted?
    4. **Completeness Check**: For "total" questions, is all relevant data included?
    5. **Category Precision**: Are asset classes vs. fund types vs. objectives correctly distinguished?
    
    ### Common Precision Errors to Avoid:
    - Confusing "mutual fund equity assets" with "total equity assets across all vehicles"
    - Mixing domestic and international data when only one was requested
    - Using partial year data when year-end was implied
    - Conflating investment objectives with asset classes
    - Missing geographic or vehicle-type qualifiers

    ## CONFIDENCE SCORING PRECISION:

    ### Confidence Level Guidelines:
    - **1.0**: Direct table lookup with exact match to question parameters
    - **0.9**: Clear chart data with minor interpolation required
    - **0.8**: Multiple consistent sources supporting same conclusion
    - **0.7**: Single good source but some scope mismatch (e.g., 2022 vs 2023 data)
    - **0.6**: Partial data requiring reasonable inference
    - **0.5**: Limited data with significant uncertainty
    - **<0.5**: Insufficient data to answer question reliably

    ### Confidence Reduction Triggers:
    - Data from different time periods than requested (-0.1 to -0.2)
    - Geographic scope mismatch (US vs global) (-0.2)
    - Fund type scope mismatch (specific vs total) (-0.1 to -0.3)
    - Conflicting data between sources (-0.3 to -0.5)
    
    ---
    
    ## QUESTION ANALYSIS:
    **User Question**: {question.strip()}
    
    **Question Type Identification**:
    - Asset allocation breakdown? Geographic analysis? Fund flow trends? Performance comparison?
    - Time-specific or trend analysis? Single category or comprehensive view?
    - Absolute values or relative percentages? Current state or historical change?
    
    ## CONTEXT BLOCKS:
    {full_context}
    
    ---
    
    ## REQUIRED OUTPUT FORMAT:
    
    DIRECT ANSWER: [Precise numerical answer with units, time period, and scope clearly specified]
    
    CONFIDENCE: [0.0-1.0 following guidelines above, with specific reasoning for score]
    
    JUSTIFICATION: [Explanation referencing specific ICI data points, noting any limitations, scope restrictions, or validation checks applied]
    
    CITED SOURCES: [2023-factbook - page X](presigned_url) [2023-factbook - page Y](presigned_url)
    
    ## CRITICAL SUCCESS METRICS:
    - Numerical precision to appropriate decimal places
    - Explicit time period and geographic scope
    - Clear distinction between asset classes, fund types, and investment objectives  
    - Comprehensive coverage when "total" or "all" is requested
    - Acknowledgment of data limitations or gaps if present
    - Applied validation checks against known ICI data patterns
    
    Your Response:
    """)

    result = complete(
        model="claude-4-sonnet",
        prompt=[{"role": "user", "content": prompt}],
        session=session,
        options=CompleteOptions(
            temperature=0.05,  # Very low for maximum precision
            max_tokens=1500,   # Conservative to avoid token limit issues
            top_p=0.9,
            guardrails=False
        ),
        stream=False
    )

    return {
        "result": "".join(result),
        "metadata": {
            "source": "TEXT",
            "num_chunks": len(retrieved_chunks)
        },
        "prompt": prompt
    }
    

def ai_complete_on_image_async(session, question, item, text_answer):
    image_file_name = item["IMAGE_FILE_NAME"]
    original_file_name = item.get("ORIGINAL_FILE_NAME", "")
    page_number = item.get("PAGE_NUMBER", "")

    # Escape for SQL
    image_file_escaped = sql_escape(image_file_name)
    original_file_escaped = sql_escape(original_file_name)
    document_metadata_escaped = sql_escape(original_file_name)
    page_metadata_escaped = sql_escape(str(page_number))
    answer_snippet_escaped = sql_escape(text_answer["result"][:2000])

    prompt = dedent(f"""
    You are an expert visual analyst specializing in ICI Investment Company Fact Book financial charts, 
    tables, and infographics. Your role is to extract precise data from visual elements and validate 
    text-based answers against actual document imagery.

    ## ICI VISUAL DATA EXPERTISE:

    ### Chart Type Recognition & Analysis:
    - **Statistical Tables**: Multi-column layouts with headers, often showing year-over-year data
      → Extract: Exact values, time periods, row/column labels, footnotes
    - **Horizontal Bar Charts**: Category comparisons or time series
      → Extract: Scale values, category labels, time periods, data values
    - **Pie Charts**: Percentage breakdowns of total allocation
      → Extract: Segment percentages, labels, total represented, time period
    - **Line Graphs**: Trend analysis over multiple years
      → Extract: Axis labels, scale, trend direction, specific data points
    - **Infographics**: Key statistics with visual emphasis
      → Extract: Highlighted numbers, comparative ratios, summary statistics

    ### ICI-Specific Visual Patterns:
    - **Asset Allocation Pies**: Typically show equity/fixed income/money market/hybrid splits
    - **Flow Charts**: Show net flows with positive/negative indicators, usually in billions
    - **Time Series**: Usually 5-10 year timeframes ending in current year (2023)
    - **Geographic Breakdowns**: US vs. International or regional distributions
    - **Fund Type Comparisons**: Mutual funds vs. ETFs vs. closed-end funds

    ### ADVANCED VISUAL INTELLIGENCE:

    #### ICI Chart Pattern Recognition:
    - **Figure Numbers**: ICI uses "Figure X.X" numbering - extract for precise citation
    - **Table Headers**: Often multi-level headers (Year, Category, Subcategory)
    - **Footnote Symbols**: *, †, ‡ indicate important qualifiers - ALWAYS check
    - **Color Coding**: Consistent colors for fund types across charts
    - **Scale Breaks**: Watch for axis breaks that might distort visual interpretation

    #### Visual Data Extraction Hierarchy:
    1. **Primary Data**: Main chart/table values (highest priority)
    2. **Footnotes**: Critical context and definitions  
    3. **Source Lines**: Data collection methodology and timing
    4. **Axis Labels**: Units, time periods, geographic scope
    5. **Legend Information**: Category definitions and color coding

    ### Visual Data Extraction Protocol:
    1. **Chart Title & Context**: What is being measured, time period, scope
    2. **Axis Labels & Scales**: Units (billions, percentages), time periods, categories
    3. **Data Values**: Precise numbers, percentages, trends
    4. **Footnotes & Qualifiers**: Important context about data scope or methodology
    5. **Visual Emphasis**: What data points are highlighted or emphasized

    ## VALIDATION TASK:

    **Original Question**: {question.strip()}
    
    **Text Answer Being Validated**:
    {text_answer["result"]}
    
    **Image Source**: Document: `{original_file_name}`, Page: {page_number}

    ## SYSTEMATIC VISUAL ANALYSIS:

    ### Step 1: Image Content Identification
    - What type of visual element is this? (table, chart, infographic, mixed)
    - What is the primary data being presented?
    - What time period and scope does it cover?
    - Are there Figure numbers or Table numbers for precise citation?
    
    ### Step 2: Precise Data Extraction
    - Extract all relevant numerical values visible in the image
    - Note units (billions, percentages, etc.), time periods, and categorical labels
    - Identify any footnotes, symbols, or qualifiers
    - Check for multi-level headers or complex data structures
    
    ### Step 3: Answer Validation
    - Compare text answer values with visual data point by point
    - Check time periods, scope, and units match exactly
    - Verify completeness - is any relevant visual data missing from text answer?
    - Assess if text answer scope aligns with visual data scope
    
    ### Step 4: Accuracy Assessment
    - Are there numerical discrepancies between text and visual?
    - Is the text answer scope too narrow or too broad for the visual data?
    - Does the text answer properly interpret the visual context and footnotes?
    - Are there additional insights in the visual that enhance the answer?

    ## ENHANCED VALIDATION CRITERIA:

    ### Numerical Precision:
    - Values match exactly or within reasonable rounding (±0.1% for percentages)
    - Units (billions/percentages/ratios) correctly interpreted
    - Time periods precisely aligned with what's shown
    - Scale factors (thousands, millions, billions) properly applied
    
    ### Scope Alignment:
    - Geographic scope (US vs. global) correctly identified from visual labels
    - Fund type coverage (all vs. specific) properly interpreted from chart context
    - Asset class vs. fund type distinction maintained per visual categorization
    - Time period coverage matches visual data timeframe
    
    ### Completeness Assessment:
    - All relevant visual data incorporated into assessment
    - No cherry-picking of convenient data points
    - Comprehensive answer when visual shows comprehensive data
    - Footnotes and qualifiers properly considered

    ### ICI-Specific Validation:
    - Asset allocation percentages sum to 100% (±1% for rounding)
    - Fund type ratios align with known ICI patterns (MF > ETF > CEF)
    - Time series show logical progression year-over-year
    - Geographic splits align with US investment patterns

    ## REQUIRED OUTPUT FORMAT:

    CRITIQUE_RESULT: [CONFIRMED/REQUIRES_CORRECTION/NEEDS_ENHANCEMENT] - [Brief assessment with specific reasoning]
    
    VISUAL_DATA_EXTRACTED: [Specific values, percentages, trends visible in image with exact figures, units, and time periods]
    
    ACCURACY_VALIDATION: [Detailed point-by-point comparison of text answer vs. visual data with specific discrepancies noted]
    
    SCOPE_ASSESSMENT: [Whether text answer scope matches visual data scope - time period, geography, fund types, completeness]
    
    FOOTNOTE_ANALYSIS: [Any footnotes, symbols, or qualifiers visible that affect interpretation]
    
    MISSING_INSIGHTS: [Any relevant data visible in image but not captured in text answer]
    
    CORRECTED_ANSWER: [If corrections needed, provide precise corrected answer based on visual data with exact values and proper context]
    
    CONFIDENCE_IN_VALIDATION: [0.0-1.0 based on clarity of visual data, completeness of extraction, and certainty of assessment]
    
    ## CRITICAL VALIDATION FOCUS:
    - ICI Fact Book visual elements are authoritative source of truth
    - Extract exact numerical values, not approximations
    - Consider footnotes and qualifiers as critical context
    - Distinguish between individual data points and totals/summaries
    - Maintain precision in temporal and geographic scope
    - Apply ICI-specific knowledge of data patterns and relationships
    
    Analysis:
    """)

    prompt_escaped = prompt.replace("'", "\\'")

    df = session.sql(f"""
        select 
            '{original_file_escaped}' as original_file_name,
            '{image_file_escaped}' as image_file_name,
            '{document_metadata_escaped}' as document_metadata,
            '{page_metadata_escaped}' as page_metadata,
            get_presigned_url('@CORTEX_SEARCH_TUTORIAL_DB.public.doc_repo', '{image_file_escaped}') as presigned_url,
            ai_complete(
                'claude-4-sonnet',
                '{prompt_escaped}',
                to_file('@CORTEX_SEARCH_TUTORIAL_DB.public.doc_repo', '{image_file_escaped}'),
                object_construct('temperature', 0.1, 'top_p', 0.9, 'max_tokens', 2500, 'guardrails', FALSE)
            ) as result
    """)
    return df.collect_nowait()


def synthesise_all_answers(session, question, text_answer_dict, image_answer_dicts):
    text_result = text_answer_dict["result"]
    text_meta = text_answer_dict.get("metadata", {})

    image_sections = []
    for img in image_answer_dicts:
        presigned_link = img.get("PRESIGNED_URL", "#")
        section = dedent(f"""
        --- 
        📄 **Source**: [{img["ORIGINAL_FILE_NAME"]}]({presigned_link})
        🖼️ Image File: `{img["IMAGE_FILE_NAME"]}`

        📘 Page Metadata:
        {img.get("PAGE_METADATA", "N/A")}

        📚 Document Metadata:
        {img.get("DOCUMENT_METADATA", "N/A")}

        📌 Visual Validation Results:
        {img["RESULT"]}
        """)
        image_sections.append(section.strip())

    image_critique_block = "\n\n".join(image_sections)

    prompt = dedent(f"""
    You are synthesizing the definitive answer to a question about 2023 ICI Investment Company Fact Book data, 
    combining text-based analysis with visual validation from actual document pages to create the most 
    accurate and authoritative response possible.

    ## SYNTHESIS OBJECTIVE:
    Create the most accurate, complete, and precise answer by integrating:
    1. Text-based analysis from enriched document chunks
    2. Visual validation from actual ICI Fact Book page images
    3. Cross-validation between multiple data sources
    4. Application of ICI-specific knowledge and data patterns

    ## USER QUESTION:
    {question}

    ## TEXT-BASED ANALYSIS:
    {text_result}

    ## VISUAL VALIDATION RESULTS:
    {image_critique_block}

    ## SYNTHESIS REQUIREMENTS:

    ### Accuracy Priority Hierarchy:
    1. **Visual data from ICI pages** (authoritative when clear and relevant)
    2. **Text analysis validated by visuals** (high confidence)
    3. **Consistent text analysis across sources** (good confidence)
    4. **Single source text analysis** (moderate confidence)
    5. **Inference from partial data** (low confidence - note limitations)

    ### Conflict Resolution Protocol:
    - If visual contradicts text: Prioritize visual data (ICI pages are source of truth)
    - If multiple visuals conflict: Note discrepancy and use most comprehensive source
    - If visual unclear: Rely on text analysis but note visual limitation
    - If both uncertain: Provide best available answer with clear confidence qualifier

    ### Precision Standards for ICI Data:
    - Maintain exact numerical values from authoritative sources
    - Specify time periods (year-end 2023, Q4 2023, etc.)
    - Include geographic scope (US, global, international)
    - Note data universe (all investment companies, specific fund types)
    - Use proper ICI terminology and category definitions
    - Include appropriate units (billions of dollars, percentages, basis points)

    ### Completeness Assessment:
    - Ensure answer fully addresses the question scope
    - Include all relevant data when comprehensive breakdown requested
    - Distinguish between asset classes, fund types, and investment objectives
    - Note if partial data or if additional context would enhance understanding
    - Address both quantitative data and qualitative insights when relevant

    ## ICI-SPECIFIC SYNTHESIS INTELLIGENCE:

    ### Data Validation Against Known Patterns:
    - Total investment company assets: ~$27+ trillion (2023)
    - Mutual funds should dominate (70-75% of total)
    - ETFs growing but still smaller (20-25% of total)
    - Closed-end funds smallest segment (1-2% of total)
    - Equity typically largest asset class (50-60%)
    - Fixed income second largest (25-35%)
    - Money market varies with market conditions (5-15%)

    ### Synthesis Quality Checks:
    - Do totals add up appropriately?
    - Are percentages reasonable within ICI context?
    - Does temporal data show logical progression?
    - Are geographic splits consistent with US investment patterns?
    - Do fund type ratios align with market structure?

    ## FINAL ANSWER REQUIREMENTS:

    **Tone**: Professional, authoritative, precise (suitable for financial analysis)
    **Structure**: 
    1. Direct answer to question with key figures
    2. Additional context and breakdowns as relevant
    3. Time period and scope qualifiers
    4. Any important limitations or notes

    **Precision**: 
    - Exact figures with proper units and context
    - Appropriate decimal places (typically 1 decimal for percentages)
    - Clear temporal and geographic qualifiers
    - Proper distinction between categories

    **Sources**: Clean citation format with working hyperlinks to actual pages

    ## ENHANCED SYNTHESIS PROTOCOL:

    ### Integration Strategy:
    1. **Start with most authoritative data** (visual validation when available)
    2. **Layer in supporting context** from text analysis
    3. **Cross-reference for consistency** across all sources
    4. **Apply ICI knowledge** for reasonableness checks
    5. **Note any limitations** or gaps in available data

    ### Output Optimization:
    - Lead with direct numerical answer when possible
    - Provide context that enhances understanding
    - Use proper financial terminology
    - Maintain professional tone suitable for investment industry
    - Include actionable insights when relevant

    ## OUTPUT FORMAT:

    [Provide comprehensive, accurate answer based on synthesis of all available data, leading with direct response to question, followed by relevant context and qualifiers]

    **Cited Sources**:
    - [2023-factbook - page X](url)
    - [2023-factbook - page Y](url)

    ## SYNTHESIS SUCCESS CRITERIA:
    - Numerical accuracy verified against visual sources when available
    - Scope precisely matches question parameters
    - Time periods and geographic context clearly specified
    - Comprehensive coverage when broad questions asked
    - Professional presentation suitable for financial analysis
    - Confidence level appropriately calibrated to data quality
    - ICI-specific patterns and knowledge properly applied

    Final Answer:
    """)

    result = complete(
        model="claude-4-sonnet",
        prompt=prompt,
        session=session,
        options=CompleteOptions(
            temperature=0.1,  # Low for precise synthesis
            max_tokens=1500,  # Conservative to avoid token limits
            top_p=0.9,
            guardrails=False
        ),
        stream=False
    )

    return "".join(result)

import re

def smart_chunk_selection(chunks, question, max_chunks=10):
    """ENHANCED HYBRID CHUNK SELECTION: Balances enriched context with raw text precision"""
    
    # ICI-specific high-value keywords with enhanced weighting
    ici_keywords = {
        'asset': 3, 'allocation': 3, 'class': 2, 'total': 3, 'net': 2,
        'equity': 2, 'fixed': 2, 'income': 2, 'money': 2, 'market': 2,
        'mutual': 2, 'fund': 2, 'etf': 2, 'exchange': 2, 'traded': 2,
        'billion': 3, 'trillion': 3, 'percentage': 2, 'breakdown': 3,
        'domestic': 2, 'international': 2, 'flow': 2, 'investment': 1,
        'company': 1, 'registered': 2, '2023': 3, '2022': 2
    }
    
    question_words = [word.lower().strip('.,!?') for word in question.split()]
    
    scored_chunks = []
    for chunk in chunks:
        # HYBRID SCORING: Consider both enriched and raw content
        enriched_text = chunk.get("ENRICHED_CHUNK", "").lower()
        raw_text = chunk.get("RAW_CHUNK_TEXT", "").lower()
        
        # Base relevance from question keywords in BOTH texts
        enriched_base = sum(3 for word in question_words if len(word) > 3 and word in enriched_text)
        raw_base = sum(4 for word in question_words if len(word) > 3 and word in raw_text)  # Higher weight for exact matches
        
        # ICI-specific scoring for both texts
        enriched_ici = sum(weight for term, weight in ici_keywords.items() if term in enriched_text)
        raw_ici = sum(weight * 1.2 for term, weight in ici_keywords.items() if term in raw_text)  # Slight boost for raw
        
        # Numerical data bonuses (more likely in raw text for exact figures)
        enriched_numerical = len(re.findall(r'\b\d+\.?\d*\b', enriched_text)) * 0.3
        raw_numerical = len(re.findall(r'\b\d+\.?\d*\b', raw_text)) * 0.8  # Higher weight for raw numbers
        
        # Percentage bonuses
        enriched_percentage = len(re.findall(r'\b\d+\.?\d*%', enriched_text)) * 0.5
        raw_percentage = len(re.findall(r'\b\d+\.?\d*%', raw_text)) * 1.2  # Raw percentages more precise
        
        # Year bonuses for both
        enriched_year = 1 if '2023' in enriched_text else (0.5 if '2022' in enriched_text else 0)
        raw_year = 2 if '2023' in raw_text else (1 if '2022' in raw_text else 0)
        
        # QUALITY BONUSES:
        # Enriched chunks with visual context get bonus
        visual_bonus = 1 if 'visual context' in enriched_text or 'chart' in enriched_text or 'table' in enriched_text else 0
        
        # Raw chunks with exact financial terms get bonus
        financial_bonus = 1 if any(term in raw_text for term in ['$', 'billion', 'trillion', 'assets', 'net']) else 0
        
        # HYBRID TOTAL SCORE
        total_score = (
            enriched_base + raw_base +
            enriched_ici + raw_ici +
            enriched_numerical + raw_numerical +
            enriched_percentage + raw_percentage +
            enriched_year + raw_year +
            visual_bonus + financial_bonus
        )
        
        scored_chunks.append((total_score, chunk))
    
    # Sort by score and take top chunks
    scored_chunks.sort(key=lambda x: x[0], reverse=True)
    
    # BALANCED SELECTION: Ensure mix of high-context and high-precision chunks
    selected_chunks = []
    enriched_heavy = 0
    raw_heavy = 0
    
    for score, chunk in scored_chunks[:max_chunks * 2]:  # Consider more candidates
        if len(selected_chunks) >= max_chunks:
            break
            
        enriched_text = chunk.get("ENRICHED_CHUNK", "")
        raw_text = chunk.get("RAW_CHUNK_TEXT", "")
        
        # Determine if chunk is enriched-heavy or raw-heavy based on content length/richness
        is_enriched_heavy = len(enriched_text) > len(raw_text) * 2
        is_raw_heavy = len(raw_text) > 100 and any(char.isdigit() for char in raw_text)
        
        # Balance selection
        if is_enriched_heavy and enriched_heavy < max_chunks * 0.6:  # Up to 60% enriched
            selected_chunks.append(chunk)
            enriched_heavy += 1
        elif is_raw_heavy and raw_heavy < max_chunks * 0.5:  # Up to 50% raw-focused
            selected_chunks.append(chunk)
            raw_heavy += 1
        elif len(selected_chunks) < max_chunks:  # Fill remaining slots
            selected_chunks.append(chunk)
    
    return selected_chunks 

## 📘 Streamlit App:  Q\&A Workflow


### 🔢 **Workflow Overview**

The pipeline consists of **7 sequential steps**, shown with real-time status updates in the UI:

---

### 1. **Document Search (Cortex Semantic Search)**

* The user enters a question via `st.chat_input()`.
* The question is normalized using `rephrase_for_search()`.
* `query_search_service()` performs an **embedding + keyword** search using the Cortex Search Service.
* Returns up to 50 enriched document chunks with metadata, image filenames, and associated PDF references.
* If no chunks are found, the assistant replies with a fallback message.

---

### 2. **Answer Generation from Text Chunks**

* `ai_complete_on_text()` receives the retrieved chunks and builds a prompt with:

  * **Clickable presigned URLs**
  * Clean markdown formatting
  * Strong precision constraints to avoid hallucinations
* The model (`claude-3-7-sonnet`) returns:

  * A **direct answer**
  * A **confidence score**
  * A **justification**
  * **CITED SOURCES** with Markdown links

---

### 3. **Image Deduplication**

* To reduce cost and noise, `(PDF_FILE_NAME, IMAGE_FILE_NAME)` pairs are deduplicated.
* This ensures each page is only submitted once for image-based validation.

---

### 4. **Extract Cited Sources & Pages**

* The model's output from Step 2 is parsed using `extract_cited_docs_and_pages()` to identify only the **relevant documents and pages** to check against image data.
* Uses regex to extract all `Document - page X` citations for focused validation.

---

### 5. **Filter Images to Cited Pages**

* The deduplicated images are **filtered** to those matching the cited documents and pages.
* Matching is done via fuzzy string matching and page number extraction.
* Each matched page is then submitted to Cortex with `ai_complete_on_image_async()` for image critique.

---

### 6. **Run Image-Based Validation (Async)**

* Cortex jobs are resolved using `resolve_async_job()` and visual progress is displayed.
* Errors are handled gracefully, and failed results are tagged with placeholders.
* The results include `RESULT`, `PRESIGNED_URL`, and other metadata fields.

---

### 7. **Final Answer Synthesis**

* All validated image responses are passed to `synthesise_all_answers()` along with the original text answer.
* The synthesis prompt:

  * Merges, reconciles, and improves factual accuracy.
  * Rephrases the result clearly for human readers.
  * **Cites each document+page with a working hyperlink**.
* The result is then displayed to the user in the chat.

---

In [ ]:
import streamlit as st
import pandas as pd
import json
import time
import asyncio
from concurrent.futures import ThreadPoolExecutor, as_completed
import re
from snowflake.core import Root

st.title("🏦 Financial Document AI Assistant")

# Initialize session state for custom questions
if 'custom_results' not in st.session_state:
    st.session_state.custom_results = {}

# Sidebar settings
with st.sidebar:
    st.markdown("## ⚙️ Performance Settings")
    st.markdown("**Image Analysis:**")
    MAX_IMAGES_TO_ANALYZE = st.slider("Max Images to Analyze", 1, 20, 8)
    st.write(f"Currently analyzing top {MAX_IMAGES_TO_ANALYZE} images")
    
    if st.button("🔄 Reset Settings"):
        st.experimental_rerun()

# User question input
user_question = st.text_input(
    "💭 Ask a question about financial documents:",
    placeholder="e.g., What are the trends in mutual fund assets?"
)

# Add submit button
col1, col2 = st.columns([3, 1])
with col2:
    submit_clicked = st.button("🔍 Ask Question", type="primary")

if user_question and submit_clicked:
    with st.spinner("🔍 Processing your question..."):
        start_time = time.time()
        
        # Step 1: Search
        st.write("🔍 Step 1 of 7: Searching vector database...")
        # Initialize the search service
        root = Root(sp_session)
        search_service = (root
            .databases["CORTEX_SEARCH_TUTORIAL_DB"]
            .schemas["PUBLIC"]
            .cortex_search_services["DOCS_SEARCH_SERVICE"]
        )
        search_results = query_multi_index_search_service(sp_session, search_service, user_question)
        
        # Debug: Check what we got from search
        st.write(f"Debug - Search results type: {type(search_results)}")
        
        # Parse JSON string to Python object if needed
        if isinstance(search_results, str):
            search_results = json.loads(search_results)
        
        # The search results likely come as a nested structure, extract the actual results
        if isinstance(search_results, dict) and 'results' in search_results:
            search_results = search_results['results']
        elif isinstance(search_results, dict) and 'data' in search_results:
            search_results = search_results['data']
        
        st.write(f"Debug - After parsing, type: {type(search_results)}, length: {len(search_results) if hasattr(search_results, '__len__') else 'N/A'}")
        
        # Step 2: Smart chunk selection
        st.write("🧠 Step 2 of 7: Smart chunk selection...")
        deduped_results = smart_chunk_selection(search_results, user_question)
        
        # Step 3: Text analysis
        st.write("📝 Step 3 of 7: Analyzing text content...")
        # Pass the full chunk objects, not just the enriched text
        answer_text = ai_complete_on_text(sp_session, user_question, deduped_results)
        
        # Step 4: Extract citations
        st.write("📚 Step 4 of 7: Extracting citations...")
        # Extract the result string from the dictionary returned by ai_complete_on_text
        answer_text_str = answer_text.get("result", "") if isinstance(answer_text, dict) else str(answer_text)
        cited_docs_pages = extract_cited_docs_and_pages(answer_text_str)
        
        # Step 5: Match images
        st.write("🖼️ Step 5 of 7: Matching relevant images...")
        
        # Smart image scoring function
        def score_image_relevance(item, question):
            """Score image relevance based on multiple factors"""
            score = 0
            question_lower = question.lower()
            content = item.get('ENRICHED_CHUNK', '').lower()
            raw_content = item.get('RAW_CHUNK_TEXT', '').lower()
            
            # Financial keywords boost
            financial_terms = ['trillion', 'billion', 'million', 'percent', '%', 'assets', 'funds', 'investment', 'expense', 'ratio', 'market', 'share']
            for term in financial_terms:
                if term in content or term in raw_content:
                    score += 10
            
            # Question keyword overlap
            question_words = set(question_lower.split())
            content_words = set(content.split())
            overlap = len(question_words.intersection(content_words))
            score += overlap * 5
            
            # Numerical content bonus
            if any(char.isdigit() for char in content):
                score += 20
            
            # Chart/table indicators
            chart_indicators = ['chart', 'table', 'figure', 'graph', 'data']
            for indicator in chart_indicators:
                if indicator in content:
                    score += 15
            
            return score
        
        # Get all available images and score them
        all_images = [result for result in deduped_results if result.get('IMAGE_FILE_NAME')]
        
        if all_images:
            # Score and sort images
            scored_images = [(item, score_image_relevance(item, user_question)) for item in all_images]
            scored_images.sort(key=lambda x: x[1], reverse=True)
            
            # Take top N images
            matched_images = [item for item, score in scored_images[:MAX_IMAGES_TO_ANALYZE]]
            
            st.write(f"Found {len(all_images)} total images, analyzing top {len(matched_images)} most relevant")
        else:
            matched_images = []
            st.write("No images found for analysis")
        
        # Step 6: Process citations and create fallback if needed
        st.write("⚙️ Step 6 of 7: Processing citations...")
        
        if not cited_docs_pages:
            st.write("⚠️ No citations found - activating fallback mode")
            # Create fallback citations from available images
            for result in matched_images[:5]:  # Limit fallback to top 5
                doc_name = result.get('ORIGINAL_FILE_NAME', 'Unknown')
                page_num = str(result.get('PAGE_NUMBER', 0))
                if doc_name not in cited_docs_pages:
                    cited_docs_pages[doc_name] = set()
                cited_docs_pages[doc_name].add(page_num)
            
            st.write(f"✅ Created fallback citations for {len(cited_docs_pages)} documents")
        
        # Step 7: Synthesize
        st.write("🧪 Step 7 of 7: Synthesize final answer")
        st.write("Synthesizing text + image answers into a final response...")
        
        # Process images with progress tracking
        image_critiques = []
        if matched_images:
            progress_placeholder = st.empty()
            
            def process_limited_images():
                """Process images synchronously using Snowpark jobs"""
                critiques = []
                
                for i, result in enumerate(matched_images):
                    progress_placeholder.text(f"Processing image critiques... ({i+1}/{len(matched_images)})")
                    
                    # Create Snowpark job for this image
                    job = ai_complete_on_image_async(sp_session, user_question, result, answer_text)
                    
                    # Resolve the job synchronously
                    resolved_result = resolve_async_job(job)
                    critique = resolved_result.get("RESULT", "") if resolved_result else ""
                    
                    if critique and critique.strip():
                        critiques.append(critique)
                
                return critiques
            
            # Run image processing
            image_critiques = process_limited_images()
            progress_placeholder.empty()
        
        # Combine text and image results
        final_answer = answer_text_str
        if image_critiques:
            combined_critique = '\n\n'.join([c for c in image_critiques if c and c.strip()])
            if combined_critique:
                final_answer += f"\n\n**Additional Image Analysis:**\n{combined_critique}"
        
        # Display final answer
        st.markdown("## 🎯 Final Answer:")
        st.markdown(final_answer)
        
        # Performance metrics
        total_time = time.time() - start_time
        st.markdown(f"⏱️ **Total time taken:** {total_time:.2f} seconds")
    
    # Debug and additional analysis sections
    with st.expander("🔧 Debug - Pipeline Diagnostics"):
        st.write("**🔍 DIAGNOSTIC - Search Results:**")
        st.write(f"Total search results: {len(search_results) if search_results else 0}")
        st.write(f"After smart selection: {len(deduped_results) if deduped_results else 0}")
        
        st.write("**🔍 DIAGNOSTIC - Extracted Citations:**")
        st.write(f"Citations found: {dict(cited_docs_pages) if cited_docs_pages else {}}")
        st.write(f"Number of cited documents: {len(cited_docs_pages) if cited_docs_pages else 0}")
        
        st.write("**🔍 DIAGNOSTIC - Text Answer Sample:**")
        st.write(f"Answer text (first 500 chars): {answer_text_str[:500]}...")
        
        st.write("**🔍 DIAGNOSTIC - Available Images:**")
        for i, result in enumerate(matched_images[:5]):
            doc_name = result.get('ORIGINAL_FILE_NAME', 'Unknown')
            img_file = result.get('IMAGE_FILE_NAME', 'Unknown')
            st.write(f"Image {i+1}: {doc_name} -> {img_file}")
        
        st.write("**🔍 DIAGNOSTIC - Final Results:**")
        st.write(f"Total jobs created: {len(image_critiques)}")
        st.write(f"Successful critiques: {len([c for c in image_critiques if c and c.strip()])}")
    
    with st.expander("🔍 Debug - Raw Image Answers"):
        if matched_images:
            st.write(f"Found {len(matched_images)} relevant images for analysis:")
            
            for i, ans in enumerate(matched_images):
                st.markdown(f"### 📄 **Image {i+1}**")
                
                # Display image metadata
                col1, col2 = st.columns([1, 1])
                with col1:
                    st.write(f"**📄 Document:** {ans.get('ORIGINAL_FILE_NAME', 'Unknown')}")
                    st.write(f"**🖼️ Image File:** {ans.get('IMAGE_FILE_NAME', 'Unknown')}")
                    st.write(f"**📄 Page:** {ans.get('PAGE_NUMBER', 'Unknown')}")
                
                with col2:
                    # Display content preview
                    content_preview = ans.get('ENRICHED_CHUNK', 'No content available')[:200]
                    st.write(f"**📝 Content Preview:** {content_preview}...")
                
                # Display the actual image
                try:
                    image_file_name = ans.get('IMAGE_FILE_NAME')
                    if image_file_name:
                        image_path = f"@CORTEX_SEARCH_TUTORIAL_DB.PUBLIC.DOC_REPO/{image_file_name}"
                        
                        try:
                            # Use get_stream() for Snowflake stage files
                            with sp_session.file.get_stream(image_path, decompress=False) as stream:
                                image_bytes = stream.read()
                            st.image(image_bytes, caption=f"Page {ans.get('PAGE_NUMBER', 'Unknown')}", width=300)
                        except Exception as e:
                            st.error(f"Could not load image: {e}")
                            st.write(f"Attempted path: {image_path}")
                            st.write(f"Debug Info:")
                            st.write(f"image_file: {ans.get('IMAGE_FILE_NAME')}")
                            st.write(f"Available ans keys: {list(ans.keys())}")
                except Exception as e:
                    st.error(f"Error processing image: {e}")
                
                # Custom question section with session state
                st.markdown("---")
                st.markdown("**🤔 Ask a custom question about this specific image:**")
                
                # Create unique keys for this image
                question_key = f"img_question_{i}"
                result_key = f"img_result_{i}"
                
                # Custom question input
                custom_question = st.text_input(
                    "Your question:",
                    key=question_key,
                    placeholder="e.g., What are the exact numbers in this chart?",
                    help="Ask specific questions about this image to get targeted analysis"
                )
                
                col_btn1, col_btn2 = st.columns([1, 1])
                
                with col_btn1:
                    # Direct analysis without page refresh
                    if st.button(f"🔍 Analyze Image", key=f"analyze_btn_{i}"):
                        if custom_question:
                            with st.spinner("🧠 Analyzing image with your question..."):
                                try:
                                    # Run AI_COMPLETE on the specific image with custom question
                                    mock_text_answer = {"result": f"Custom question: {custom_question}"}
                                    job = ai_complete_on_image_async(sp_session, custom_question, ans, mock_text_answer)
                                    resolved_result = resolve_async_job(job)
                                    custom_critique = resolved_result.get("RESULT", "") if resolved_result else ""
                                    
                                    # Store result in session state
                                    st.session_state[result_key] = custom_critique
                                    st.success("Analysis complete! Check result below.")
                                    
                                except Exception as e:
                                    st.error(f"Error analyzing image: {e}")
                        else:
                            st.warning("Please enter a question first.")
                
                # Display stored result if available
                if result_key in st.session_state and st.session_state[result_key]:
                    st.success("**🎯 Custom AI Analysis:**")
                    st.markdown(st.session_state[result_key])
                
                with col_btn2:
                    # Clear result button
                    if st.button(f"🗑️ Clear", key=f"clear_btn_{i}"):
                        # Clear stored results for this image
                        if result_key in st.session_state:
                            del st.session_state[result_key]
                        st.success("Result cleared!")
                
                st.markdown("---")
        else:
            st.write("No images available for analysis.")
    
    with st.expander("📊 Debug - Hybrid Text Analysis"):
        if deduped_results:
            st.write("**Enriched vs Raw Content Comparison:**")
            for i, chunk in enumerate(deduped_results[:5]):  # Show first 5 chunks
                st.markdown(f"### Chunk {i+1}")
                
                col1, col2 = st.columns(2)
                with col1:
                    st.markdown("**🤖 Enriched Content:**")
                    enriched = chunk.get('ENRICHED_CHUNK', 'N/A')[:300]
                    st.text_area("", enriched, height=100, key=f"enriched_{i}", disabled=True)
                
                with col2:
                    st.markdown("**📄 Raw Content:**") 
                    raw = chunk.get('RAW_CHUNK_TEXT', 'N/A')[:300]
                    st.text_area("", raw, height=100, key=f"raw_{i}", disabled=True)
                
                st.markdown("---")

st.markdown("---")

Give me a break down of the company's total net investments by percentage of asset class. 


In [ ]:
import streamlit as st
import pandas as pd
import json
import time
import asyncio
from concurrent.futures import ThreadPoolExecutor, as_completed
import re
from snowflake.core import Root

st.title("🏦 Financial Document AI Assistant")

# Initialize session state for custom questions
if 'custom_results' not in st.session_state:
    st.session_state.custom_results = {}

# Sidebar settings
with st.sidebar:
    st.markdown("## ⚙️ Performance Settings")
    st.markdown("**Image Analysis:**")
    MAX_IMAGES_TO_ANALYZE = st.slider("Max Images to Analyze", 1, 20, 8)
    st.write(f"Currently analyzing top {MAX_IMAGES_TO_ANALYZE} images")
    
    if st.button("🔄 Reset Settings"):
        st.experimental_rerun()

# User question input
user_question = st.text_input(
    "💭 Ask a question about financial documents:",
    placeholder="e.g., What are the trends in mutual fund assets?"
)

# Add submit button
col1, col2 = st.columns([3, 1])
with col2:
    submit_clicked = st.button("🔍 Ask Question", type="primary")

if user_question and submit_clicked:
    with st.spinner("🔍 Processing your question..."):
        start_time = time.time()
        
        # Step 1: Search
        st.write("🔍 Step 1 of 7: Searching vector database...")
        # Initialize the search service
        root = Root(sp_session)
        search_service = (root
            .databases["CORTEX_SEARCH_TUTORIAL_DB"]
            .schemas["PUBLIC"]
            .cortex_search_services["DOCS_SEARCH_SERVICE"]
        )
        search_results = query_multi_index_search_service(sp_session, search_service, user_question)
        
        # Debug: Check what we got from search
        st.write(f"Debug - Search results type: {type(search_results)}")
        
        # Parse JSON string to Python object if needed
        if isinstance(search_results, str):
            search_results = json.loads(search_results)
        
        # The search results likely come as a nested structure, extract the actual results
        if isinstance(search_results, dict) and 'results' in search_results:
            search_results = search_results['results']
        elif isinstance(search_results, dict) and 'data' in search_results:
            search_results = search_results['data']
        
        st.write(f"Debug - After parsing, type: {type(search_results)}, length: {len(search_results) if hasattr(search_results, '__len__') else 'N/A'}")
        
        # Step 2: Smart chunk selection
        st.write("🧠 Step 2 of 7: Smart chunk selection...")
        deduped_results = smart_chunk_selection(search_results, user_question)
        
        # Step 3: Text analysis
        st.write("📝 Step 3 of 7: Analyzing text content...")
        # Pass the full chunk objects, not just the enriched text
        answer_text = ai_complete_on_text(sp_session, user_question, deduped_results)
        
        # Step 4: Extract citations
        st.write("📚 Step 4 of 7: Extracting citations...")
        # Extract the result string from the dictionary returned by ai_complete_on_text
        answer_text_str = answer_text.get("result", "") if isinstance(answer_text, dict) else str(answer_text)
        cited_docs_pages = extract_cited_docs_and_pages(answer_text_str)
        
        # Step 5: Match images
        st.write("🖼️ Step 5 of 7: Matching relevant images...")
        
        # Smart image scoring function
        def score_image_relevance(item, question):
            """Score image relevance based on multiple factors"""
            score = 0
            question_lower = question.lower()
            content = item.get('ENRICHED_CHUNK', '').lower()
            raw_content = item.get('RAW_CHUNK_TEXT', '').lower()
            
            # Financial keywords boost
            financial_terms = ['trillion', 'billion', 'million', 'percent', '%', 'assets', 'funds', 'investment', 'expense', 'ratio', 'market', 'share']
            for term in financial_terms:
                if term in content or term in raw_content:
                    score += 10
            
            # Question keyword overlap
            question_words = set(question_lower.split())
            content_words = set(content.split())
            overlap = len(question_words.intersection(content_words))
            score += overlap * 5
            
            # Numerical content bonus
            if any(char.isdigit() for char in content):
                score += 20
            
            # Chart/table indicators
            chart_indicators = ['chart', 'table', 'figure', 'graph', 'data']
            for indicator in chart_indicators:
                if indicator in content:
                    score += 15
            
            return score
        
        # Get all available images and score them
        all_images = [result for result in deduped_results if result.get('IMAGE_FILE_NAME')]
        
        if all_images:
            # Score and sort images
            scored_images = [(item, score_image_relevance(item, user_question)) for item in all_images]
            scored_images.sort(key=lambda x: x[1], reverse=True)
            
            # Take top N images
            matched_images = [item for item, score in scored_images[:MAX_IMAGES_TO_ANALYZE]]
            
            st.write(f"Found {len(all_images)} total images, analyzing top {len(matched_images)} most relevant")
        else:
            matched_images = []
            st.write("No images found for analysis")
        
        # Step 6: Process citations and create fallback if needed
        st.write("⚙️ Step 6 of 7: Processing citations...")
        
        if not cited_docs_pages:
            st.write("⚠️ No citations found - activating fallback mode")
            # Create fallback citations from available images
            for result in matched_images[:5]:  # Limit fallback to top 5
                doc_name = result.get('ORIGINAL_FILE_NAME', 'Unknown')
                page_num = str(result.get('PAGE_NUMBER', 0))
                if doc_name not in cited_docs_pages:
                    cited_docs_pages[doc_name] = set()
                cited_docs_pages[doc_name].add(page_num)
            
            st.write(f"✅ Created fallback citations for {len(cited_docs_pages)} documents")
        
        # Step 7: Synthesize
        st.write("🧪 Step 7 of 7: Synthesize final answer")
        st.write("Synthesizing text + image answers into a final response...")
        
        # Process images with progress tracking
        image_critiques = []
        if matched_images:
            progress_placeholder = st.empty()
            
            def process_limited_images():
                """Process images synchronously using Snowpark jobs"""
                critiques = []
                
                for i, result in enumerate(matched_images):
                    progress_placeholder.text(f"Processing image critiques... ({i+1}/{len(matched_images)})")
                    
                    # Create Snowpark job for this image
                    job = ai_complete_on_image_async(sp_session, user_question, result, answer_text)
                    
                    # Resolve the job synchronously
                    resolved_result = resolve_async_job(job)
                    critique = resolved_result.get("RESULT", "") if resolved_result else ""
                    
                    if critique and critique.strip():
                        critiques.append(critique)
                
                return critiques
            
            # Run image processing
            image_critiques = process_limited_images()
            progress_placeholder.empty()
        
        # Combine text and image results
        final_answer = answer_text_str
        if image_critiques:
            combined_critique = '\n\n'.join([c for c in image_critiques if c and c.strip()])
            if combined_critique:
                final_answer += f"\n\n**Additional Image Analysis:**\n{combined_critique}"
        
        # Display final answer
        st.markdown("## 🎯 Final Answer:")
        st.markdown(final_answer)
        
        # Performance metrics
        total_time = time.time() - start_time
        st.markdown(f"⏱️ **Total time taken:** {total_time:.2f} seconds")
    
    # Debug and additional analysis sections
    with st.expander("🔧 Debug - Pipeline Diagnostics"):
        st.write("**🔍 DIAGNOSTIC - Search Results:**")
        st.write(f"Total search results: {len(search_results) if search_results else 0}")
        st.write(f"After smart selection: {len(deduped_results) if deduped_results else 0}")
        
        st.write("**🔍 DIAGNOSTIC - Extracted Citations:**")
        st.write(f"Citations found: {dict(cited_docs_pages) if cited_docs_pages else {}}")
        st.write(f"Number of cited documents: {len(cited_docs_pages) if cited_docs_pages else 0}")
        
        st.write("**🔍 DIAGNOSTIC - Text Answer Sample:**")
        st.write(f"Answer text (first 500 chars): {answer_text_str[:500]}...")
        
        st.write("**🔍 DIAGNOSTIC - Available Images:**")
        for i, result in enumerate(matched_images[:5]):
            doc_name = result.get('ORIGINAL_FILE_NAME', 'Unknown')
            img_file = result.get('IMAGE_FILE_NAME', 'Unknown')
            st.write(f"Image {i+1}: {doc_name} -> {img_file}")
        
        st.write("**🔍 DIAGNOSTIC - Final Results:**")
        st.write(f"Total jobs created: {len(image_critiques)}")
        st.write(f"Successful critiques: {len([c for c in image_critiques if c and c.strip()])}")
    
    with st.expander("🔍 Debug - Raw Image Answers"):
        if matched_images:
            st.write(f"Found {len(matched_images)} relevant images for analysis:")
            
            for i, ans in enumerate(matched_images):
                st.markdown(f"### 📄 **Image {i+1}**")
                
                # Display image metadata
                col1, col2 = st.columns([1, 1])
                with col1:
                    st.write(f"**📄 Document:** {ans.get('ORIGINAL_FILE_NAME', 'Unknown')}")
                    st.write(f"**🖼️ Image File:** {ans.get('IMAGE_FILE_NAME', 'Unknown')}")
                    st.write(f"**📄 Page:** {ans.get('PAGE_NUMBER', 'Unknown')}")
                
                with col2:
                    # Display content preview
                    content_preview = ans.get('ENRICHED_CHUNK', 'No content available')[:200]
                    st.write(f"**📝 Content Preview:** {content_preview}...")
                
                # Display the actual image
                try:
                    image_file_name = ans.get('IMAGE_FILE_NAME')
                    if image_file_name:
                        image_path = f"@CORTEX_SEARCH_TUTORIAL_DB.PUBLIC.DOC_REPO/{image_file_name}"
                        
                        try:
                            # Use get_stream() for Snowflake stage files
                            with sp_session.file.get_stream(image_path, decompress=False) as stream:
                                image_bytes = stream.read()
                            st.image(image_bytes, caption=f"Page {ans.get('PAGE_NUMBER', 'Unknown')}", width=300)
                        except Exception as e:
                            st.error(f"Could not load image: {e}")
                            st.write(f"Attempted path: {image_path}")
                            st.write(f"Debug Info:")
                            st.write(f"image_file: {ans.get('IMAGE_FILE_NAME')}")
                            st.write(f"Available ans keys: {list(ans.keys())}")
                except Exception as e:
                    st.error(f"Error processing image: {e}")
                
                # Custom question section with session state
                st.markdown("---")
                st.markdown("**🤔 Ask a custom question about this specific image:**")
                
                # Create unique keys for this image
                question_key = f"img_question_{i}"
                result_key = f"img_result_{i}"
                
                # Custom question input
                custom_question = st.text_input(
                    "Your question:",
                    key=question_key,
                    placeholder="e.g., What are the exact numbers in this chart?",
                    help="Ask specific questions about this image to get targeted analysis"
                )
                
                col_btn1, col_btn2 = st.columns([1, 1])
                
                with col_btn1:
                    # Direct analysis without page refresh
                    if st.button(f"🔍 Analyze Image", key=f"analyze_btn_{i}"):
                        if custom_question:
                            with st.spinner("🧠 Analyzing image with your question..."):
                                try:
                                    # Run AI_COMPLETE on the specific image with custom question
                                    mock_text_answer = {"result": f"Custom question: {custom_question}"}
                                    job = ai_complete_on_image_async(sp_session, custom_question, ans, mock_text_answer)
                                    resolved_result = resolve_async_job(job)
                                    custom_critique = resolved_result.get("RESULT", "") if resolved_result else ""
                                    
                                    # Store result in session state
                                    st.session_state[result_key] = custom_critique
                                    st.success("Analysis complete! Check result below.")
                                    
                                except Exception as e:
                                    st.error(f"Error analyzing image: {e}")
                        else:
                            st.warning("Please enter a question first.")
                
                # Display stored result if available
                if result_key in st.session_state and st.session_state[result_key]:
                    st.success("**🎯 Custom AI Analysis:**")
                    st.markdown(st.session_state[result_key])
                
                with col_btn2:
                    # Clear result button
                    if st.button(f"🗑️ Clear", key=f"clear_btn_{i}"):
                        # Clear stored results for this image
                        if result_key in st.session_state:
                            del st.session_state[result_key]
                        st.success("Result cleared!")
                
                st.markdown("---")
        else:
            st.write("No images available for analysis.")
    
    with st.expander("📊 Debug - Hybrid Text Analysis"):
        if deduped_results:
            st.write("**Enriched vs Raw Content Comparison:**")
            for i, chunk in enumerate(deduped_results[:5]):  # Show first 5 chunks
                st.markdown(f"### Chunk {i+1}")
                
                col1, col2 = st.columns(2)
                with col1:
                    st.markdown("**🤖 Enriched Content:**")
                    enriched = chunk.get('ENRICHED_CHUNK', 'N/A')[:300]
                    st.text_area("", enriched, height=100, key=f"enriched_{i}", disabled=True)
                
                with col2:
                    st.markdown("**📄 Raw Content:**") 
                    raw = chunk.get('RAW_CHUNK_TEXT', 'N/A')[:300]
                    st.text_area("", raw, height=100, key=f"raw_{i}", disabled=True)
                
                st.markdown("---")

st.markdown("---")

what is the expense ratio trends in equity for actively managed mutual funds?